<a href="https://colab.research.google.com/github/nsy033/CS376_TEAM16/blob/master/Team16_Final_MusiCNN_result_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##***CS376* Machine Learning Final Project**

**Title** |  Music mood tagging model with multi label classification using music feature data and pre-labeled data

----

    Team 16
    20190432 윤태양 Taeyang Yoon
    20190606 정재령 Jaeryung Chung
    20190656 최승연 Seungyeon Choi

###Basic Setup

#####Install required packages.

In [ ]:
# Check Colab System Spec.
!cat /etc/issue.net       # Colab OS
!head /proc/cpuinfo       # Colab CPU
!head -n 3 /proc/meminfo  # Memory Spec.
!df -h                    # Disk Spec.
!pwd                      # Current Directory

Ubuntu 18.04.5 LTS
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
MemTotal:       26692024 kB
MemFree:        24189692 kB
MemAvailable:   25779216 kB
Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   39G  188G  17% /
tmpfs            64M     0   64M   0% /dev
shm              13G     0   13G   0% /dev/shm
/dev/root       2.0G  1.2G  812M  59% /sbin/docker-init
tmpfs            13G   28K   13G   1% /var/colab
/dev/sda1       233G   41G  192G  18% /etc/hosts
tmpfs            13G     0   13G   0% /proc/acpi
tmpfs            13G     0   13G   0% /proc/scsi
tmpfs            13G     0   13G   0% /sys/firmware
/content


In [ ]:
# Install Required packages
!pip install audioread
!pip install essentia
!pip install fire
!pip install grpcio
!pip install h5py
!pip install librosa
!pip install llvmlite
!pip install matplotlib
!pip install numba
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install scipy
!pip install six
!pip install SoundFile
!pip install sox
!pip install tensorboard
!pip install tensorflow-estimator
!pip install termcolor
!pip install torch
!pip install torchaudio
!pip install tqdm

!apt install ffmpeg
!pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.6 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.0 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=5b76198014dfd62832b9d8a1f713f02a9509ce589b7784addaa125b6ce241aa7
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in

#####Import necessary libraries.

In [ ]:
# Setup Dependency
import os
import sys
import numpy as np
import pandas as pd
import zipfile
import math
import matplotlib.pyplot as plt
import random
import collections
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from scipy.special import softmax
from scipy.stats import rankdata

import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

# preprocessing/mtat_read.py
import glob
import librosa
import fire
import tqdm

# training/main.py
import argparse
import time
import datetime
import tqdm
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from torch.utils.tensorboard import SummaryWriter

# training/data_loader
import pickle
import csv
from torch.utils import data
from sklearn.preprocessing import LabelBinarizer

# prediction
from IPython.display import Audio, display, clear_output
from scipy.io import wavfile
from dl_colab_notebooks.audio import record_audio, upload_audio

###Source Data Setup

##### Methods for mount and download

In [ ]:
from google.colab import drive
import os
import gdown

def mount_drive(b):
  google_drive_prefix = '/content/drive'
  drive.mount(google_drive_prefix)

  !ls /content/drive/

  folder_directory = '/content/drive/MyDrive/CS376/'

  if not os.path.exists(folder_directory):
    os.mkdir(folder_directory)

  print('Great Job!!!' if os.path.exists(folder_directory) else 'Something wrong. The folder has not yet been created')

def download_src(dataset):
  # https://drive.google.com/uc?id= >>>file id<<<
  if dataset == "audioautix(836)":
    # data-836-001.zip : https://drive.google.com/file/d/1Hr6RkVFlfKoTKw8CqPbjVG-Exe5M5XVW/view?usp=sharing
    # data-836-002.zip : https://drive.google.com/file/d/1lDNASZXh62TM2Z3vN7Rcmf3b0yCVkwmD/view?usp=sharing
    # data-836-003.zip : https://drive.google.com/file/d/1sNai5fRjJE1_8DVlBjLns7NjdtheufDX/view?usp=sharing

    if not os.path.exists(save_path):
      os.makedirs(save_path)
      # !cd /content/drive/MyDrive/
      !gdown https://drive.google.com/uc?id=1Hr6RkVFlfKoTKw8CqPbjVG-Exe5M5XVW
      !gdown https://drive.google.com/uc?id=1lDNASZXh62TM2Z3vN7Rcmf3b0yCVkwmD
      !gdown https://drive.google.com/uc?id=1sNai5fRjJE1_8DVlBjLns7NjdtheufDX
      # print("appending zip files...")
      # os.system('cat {}/audioautix.zip.* > {}/MyDrive.zip'.format(zip_path, zip_path))
      print("unzipping data-836-001.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-836-001.zip'), save_path))
      print("unzipping data-836-002.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-836-002.zip'), save_path))
      print("unzipping data-836-003.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-836-003.zip'), save_path))
      print("Done!!")

  elif dataset == "audioautix+fma(1136, only 46)":
    # data-1136-001.zip : https://drive.google.com/file/d/15tV1N9wqn4mtRuhHaBAZu3HhFXX3FXZP/view?usp=sharing
    # data-1136-002.zip : https://drive.google.com/file/d/1Ko8wAVQy2dMnO7cgUTt8gaXJl3Ww2EzL/view?usp=sharing
    # data-1136-003.zip : https://drive.google.com/file/d/1JOOgEtLOQ4JmHjILTPFCgb6Z8la-VepI/view?usp=sharing
    # data-1136-004.zip : https://drive.google.com/file/d/1d1tK3wi2A4106tNw0ne_uGHAaSd-tt9-/view?usp=sharing

    if not os.path.exists(save_path):
      os.makedirs(save_path)
      # !cd /content/drive/MyDrive/
      !gdown https://drive.google.com/uc?id=15tV1N9wqn4mtRuhHaBAZu3HhFXX3FXZP
      !gdown https://drive.google.com/uc?id=1Ko8wAVQy2dMnO7cgUTt8gaXJl3Ww2EzL
      !gdown https://drive.google.com/uc?id=1JOOgEtLOQ4JmHjILTPFCgb6Z8la-VepI
      !gdown https://drive.google.com/uc?id=1d1tK3wi2A4106tNw0ne_uGHAaSd-tt9-
      # print("appending zip files...")
      # os.system('cat {}/audioautix.zip.* > {}/MyDrive.zip'.format(zip_path, zip_path))
      print("unzipping data-1136-001.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-1136-001.zip'), save_path))
      print("unzipping data-1136-002.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-1136-002.zip'), save_path))
      print("unzipping data-1136-003.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-1136-003.zip'), save_path))
      print("unzipping data-1136-004.zip...")
      os.system('unzip {} -d {}'.format(os.path.join(zip_path, 'data-1136-004.zip'), save_path))
      print("Done!!")
  else :
    print("Sorry, but not ready for other dataset");

#####Mount your google drive, or download pre-made execution environment by our team.

In [ ]:
setup = "download" #@param ["mount", "download"]
dataset = "audioautix(836)" #@param ["audioautix(836)", "audioautix+fma(1136, only 46)"]

zip_path = '/content/'
save_path = os.path.join(zip_path, 'drive', 'MyDrive')

if setup == "mount": 
  mount_drive("")
else:
  download_src(dataset)

Downloading...
From: https://drive.google.com/uc?id=1Hr6RkVFlfKoTKw8CqPbjVG-Exe5M5XVW
To: /content/data-836-001.zip
100% 1.24G/1.24G [00:10<00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lDNASZXh62TM2Z3vN7Rcmf3b0yCVkwmD
To: /content/data-836-002.zip
100% 1.63G/1.63G [00:20<00:00, 78.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sNai5fRjJE1_8DVlBjLns7NjdtheufDX
To: /content/data-836-003.zip
100% 1.64G/1.64G [00:16<00:00, 102MB/s]
unzipping data-836-001.zip...
unzipping data-836-002.zip...
unzipping data-836-003.zip...
Done!!


###Preprocess data

Preprocess the audio data, the wav format is recommended because of the resulting npy file size.

If you already have processed npy file in your drive, you can skip this stage.

In [ ]:
# Preprocessing
# preprocessing/mtat_read.py
class Processor:
  def __init__(self):
    self.fs = 16000
    
  def get_paths(self, data_path):
    self.files = glob.glob(os.path.join(data_path, '*.wav'))
    # print(data_path)
    self.npy_path = os.path.join(data_path, 'mtat', 'npy')
    if not os.path.exists(self.npy_path):
      os.makedirs(self.npy_path)
   
  def get_npy(self, fn):
    x, sr = librosa.core.load(fn, sr = self.fs)
    return x
    
  def iterate(self, data_path):
    iter_cnt = 0
    self.get_paths(data_path)
    for fn in tqdm.tqdm(self.files):
      npy_fn = os.path.join(self.npy_path, fn.split('/')[-1][:-3]+'npy')
      if not os.path.exists(npy_fn):
        try:
          x = self.get_npy(fn)
          print(npy_fn)
          np.save(open(npy_fn, 'wb'), x)
        except RuntimeError:
          # some audio files are broken
          print(fn)
          continue

p = Processor()
directory = '/content/drive/MyDrive/CS376/music_src/audionautix/'
p.iterate(directory)

  0%|          | 1/836 [00:02<27:54,  2.01s/it]

/content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/HorrorMusic_29_58.npy


  0%|          | 2/836 [00:03<21:46,  1.57s/it]

/content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/RedwoodHighway_58_87.npy


  0%|          | 3/836 [00:04<19:48,  1.43s/it]

...

/content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/MarathonMan.npy


100%|█████████▉| 835/836 [17:47<00:01,  1.27s/it]

/content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/RiverMeditation.npy


100%|██████████| 836/836 [17:48<00:00,  1.28s/it]

/content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/TheMightyKingdom_29_58.npy


###Modules

Some module classes that are needed for the following models.

In [ ]:
# training/modules.py
class Conv_1d(nn.Module):
    def __init__(self, input_channels, output_channels, shape=3, stride=1, pooling=2):
        super(Conv_1d, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, shape, stride=stride, padding=shape//2)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()
        self.mp = nn.MaxPool1d(pooling)
    def forward(self, x):
        out = self.mp(self.relu(self.bn(self.conv(x))))
        return out

class Conv_2d(nn.Module):
    def __init__(self, input_channels, output_channels, shape=3, stride=1, pooling=2):
        super(Conv_2d, self).__init__()
        self.conv = nn.Conv2d(input_channels, output_channels, shape, stride=stride, padding=shape//2)
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()
        self.mp = nn.MaxPool2d(pooling)
    def forward(self, x):
        out = self.mp(self.relu(self.bn(self.conv(x))))
        return out

class Conv_V(nn.Module):
    # vertical convolution
    def __init__(self, input_channels, output_channels, filter_shape):
        super(Conv_V, self).__init__()
        self.conv = nn.Conv2d(input_channels, output_channels, filter_shape,
                              padding=(0, filter_shape[1]//2))
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))
        freq = x.size(2)
        out = nn.MaxPool2d((freq, 1), stride=(freq, 1))(x)
        out = out.squeeze(2)
        return out


class Conv_H(nn.Module):
    # horizontal convolution
    def __init__(self, input_channels, output_channels, filter_length):
        super(Conv_H, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, filter_length,
                              padding=filter_length//2)
        self.bn = nn.BatchNorm1d(output_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        freq = x.size(2)
        out = nn.AvgPool2d((freq, 1), stride=(freq, 1))(x)
        out = out.squeeze(2)
        out = self.relu(self.bn(self.conv(out)))
        return out

###Model


#####Before moving on, don't forget to choose the model and specify the total number of tags first

In [ ]:
TAG_NUM =  30#@param {type:"number"}
MODEL = 'musicnn' #@param ["crnn", "musicnn"]

#####Now you can run the code!

In [ ]:
# training/model.py
SEED = 4
if MODEL == "musicnn":
  SEED = 2
np.random.seed(SEED)
class CRNN(nn.Module):
    '''
    Choi et al. 2017
    Convolution recurrent neural networks for music classification.
    Feature extraction with CNN + temporal summary with RNN
    '''
    def __init__(self,
                sample_rate=16000,
                n_fft=512,
                f_min=0.0,
                f_max=8000.0,
                n_mels=128,
                n_class=TAG_NUM):
        super(CRNN, self).__init__()

        # Spectrogram
        self.spec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate,
                                                         n_fft=n_fft,
                                                         f_min=f_min,
                                                         f_max=f_max,
                                                         n_mels=n_mels)
        self.to_db = torchaudio.transforms.AmplitudeToDB()
        self.spec_bn = nn.BatchNorm2d(1)

        # CNN
        self.layer1 = Conv_2d(1, 64, pooling=(2,2))
        self.layer2 = Conv_2d(64, 128, pooling=(3,3))
        self.layer3 = Conv_2d(128, 128, pooling=(4,4))
        self.layer4 = Conv_2d(128, 128, pooling=(4,4))

        # RNN
        self.layer5 = nn.GRU(128, 64, 2, batch_first=True)

        # Dense
        self.dropout = nn.Dropout(0.1)
        self.dense = nn.Linear(64, TAG_NUM)

    def forward(self, x):
        # Spectrogram
        x = self.spec(x)
        x = self.to_db(x)
        x = x.unsqueeze(1)
        x = self.spec_bn(x)

        # CCN
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # RNN
        x = x.squeeze(2)
        x = x.permute(0, 2, 1)
        x, _ = self.layer5(x)
        x = x[:, -1, :]

        # Dense
        x = self.dropout(x)
        x = self.dense(x)
        x = nn.Sigmoid()(x)

        return x


class Musicnn(nn.Module):
    '''
    Pons et al. 2017
    End-to-end learning for music audio tagging at scale.
    This is the updated implementation of the original paper. Referred to the Musicnn code.
    https://github.com/jordipons/musicnn
    '''
    def __init__(self,
                sample_rate=16000,
                n_fft=512,
                f_min=0.0,
                f_max=8000.0,
                n_mels=96,
                n_class=TAG_NUM,
                dataset='mtat'):
        super(Musicnn, self).__init__()

        # Spectrogram
        self.spec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate,
                                                         n_fft=n_fft,
                                                         f_min=f_min,
                                                         f_max=f_max,
                                                         n_mels=n_mels)
        self.to_db = torchaudio.transforms.AmplitudeToDB()
        self.spec_bn = nn.BatchNorm2d(1)

        # Pons front-end
        m1 = Conv_V(1, 204, (int(0.7*96), 7))
        m2 = Conv_V(1, 204, (int(0.4*96), 7))
        m3 = Conv_H(1, 51, 129)
        m4 = Conv_H(1, 51, 65)
        m5 = Conv_H(1, 51, 33)
        self.layers = nn.ModuleList([m1, m2, m3, m4, m5])

        # Pons back-end
        backend_channel= 64
        self.layer1 = Conv_1d(561, backend_channel, 7, 1, 1)
        self.layer2 = Conv_1d(backend_channel, backend_channel, 7, 1, 1)
        self.layer3 = Conv_1d(backend_channel, backend_channel, 7, 1, 1)
        self.layer4 = Conv_1d(backend_channel, backend_channel, 7, 1, 1)

        # Dense
        dense_channel = 200
        self.dense1 = nn.Linear((561+(backend_channel*4))*2, dense_channel)
        self.bn = nn.BatchNorm1d(dense_channel)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.dense2 = nn.Linear(dense_channel, n_class)

    def forward(self, x):
        # Spectrogram
        x = self.spec(x)
        x = self.to_db(x)
        x = x.unsqueeze(1)
        x = self.spec_bn(x)

        # Pons front-end
        out = []
        for layer in self.layers:
            out.append(layer(x))
        out = torch.cat(out, dim=1)

        # Pons back-end
        length = out.size(2)
        res1 = self.layer1(out)
        res2 = self.layer2(res1) + res1
        res3 = self.layer3(res2) + res2
        res4 = self.layer4(res3) + res3
        out = torch.cat([out, res1, res2, res3, res4], 1)

        mp = nn.MaxPool1d(length)(out)
        avgp = nn.AvgPool1d(length)(out)

        out = torch.cat([mp, avgp], dim=1)
        out = out.squeeze(2)

        out = self.relu(self.bn(self.dense1(out)))
        out = self.dropout(out)
        out = self.dense2(out)
        out = nn.Sigmoid()(out)

        return out

In [ ]:
from scipy.stats import rankdata

# training/solver.py
if TAG_NUM == 30:
  TAGS = ['calming', 'relaxing', 'uplifting', 'bright', 'grooving', 'energy', 'sad',
          'driving', 'bouncy', 'dark', 'pensive', 'somber', 'epic', 'cool',
          'guest-composer', 'groove', 'meloncholy', 'mysterious', 'moody', 'meditative',
          'intense', 'humorous', 'short-bits', 'suspenseful', 'christmas', 'action',
          'aggressive', 'meditation', 'melancholy', 'dramatic', 'vocal', 'dance',
          'soothing', 'light', 'happy', 'angry', 'ominous', 'evil', 'inspiration',
          'bird-house-music', 'goofy', 'silly', 'smooth', 'chill', 'bossa-nova', 'jazz']
else:
  TAGS = ['calming' 'relaxing' 'uplifting' 'bright' 'grooving' 'energy' 'sad'
          'driving' 'bouncy' 'dark' 'pensive' 'somber' 'epic' 'cool' 'melancholy'
          'mysterious' 'moody' 'meditation' 'intense' 'humorous' 'suspenseful'
          'christmas' 'action' 'aggressive' 'dramatic' 'soothing' 'happy' 'angry'
          'ominous' 'jazz']

def read_file(tsv_file):
    tracks = {}
    with open(tsv_file) as fp:
        reader = csv.reader(fp, delimiter='\t')
        next(reader, None)  # skip header
        for row in reader:
            track_id = row[0]
            tracks[track_id] = {
                'path': row[3].replace('.\wav', '.npy'),
                'tags': row[5:],
            }
    return tracks

class Solver(object):
    def __init__(self, data_loader, config):
        # data loader
        self.data_loader = data_loader
        self.dataset = config["dataset"]
        self.data_path = config["data_path"]
        self.input_length = config["input_length"]

        # training settings
        self.n_epochs = config["n_epochs"]
        self.lr = config["lr"]
        self.use_tensorboard = config["use_tensorboard"]

        # model path and step size
        self.model_save_path = config["model_save_path"]
        self.model_load_path = config["model_load_path"]
        self.log_step = config["log_step"]
        self.batch_size = config["batch_size"]
        self.model_type = config["model_type"]

        # Build model
        self.get_dataset()
        self.build_model()

        # Tensorboard
        self.writer = SummaryWriter()

        self.file_count = 0

    def get_dataset(self):
        if self.dataset == 'mtat':
          if TAG_NUM == 30:
            self.train_list = np.load('./../split/mtat/30_train.npy')
            self.valid_list = np.load('./../split/mtat/30_valid.npy')
            self.binary = np.load('./../split/mtat/30_binary.npy')
          else:
            self.train_list = np.load('./../split/mtat/46_train.npy')
            self.valid_list = np.load('./../split/mtat/46_valid.npy')
            self.binary = np.load('./../split/mtat/46_binary.npy')
            # self.train_list = np.load('./../split/mtat/train.npy')
            # self.valid_list = np.load('./../split/mtat/valid.npy')
            # self.binary = np.load('./../split/mtat/binary.npy')

    def get_model(self):
        if self.model_type == 'crnn':
            return crnn
        elif self.model_type == 'musicnn':
            return musicnn

    def build_model(self):
        # model
        self.model = self.get_model()

        # load pretrained model
        if len(self.model_load_path) > 1:
            self.load(self.model_load_path)

        # optimizers
        self.optimizer = torch.optim.Adam(self.model.parameters(), self.lr, weight_decay=1e-4)

    def load(self, filename):
        S = torch.load(filename)
        if 'spec.mel_scale.fb' in S.keys():
            self.model.spec.mel_scale.fb = S['spec.mel_scale.fb']
        self.model.load_state_dict(S)

    def to_var(self, x):
        return Variable(x)

    def get_loss_function(self):
        # return nn.BCELoss()
        return nn.BCEWithLogitsLoss()

    def train(self):
        # Start training
        print("start training with %s model..." %(self.model_type))
        start_t = time.time()
        reconst_loss = self.get_loss_function()
        best_metric = 0
        drop_counter = 0

        # Iterate
        for epoch in range(self.n_epochs):
            ctr = 0
            drop_counter += 1
            self.model = self.model.train()
            for x, y in self.data_loader:
                ctr += 1
                # Forward
                x = self.to_var(x)
                y = self.to_var(y)
                out = self.model(x)

                # Backward
                loss = reconst_loss(out, y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
            self.writer.add_scalar('Loss/train', loss.item(), epoch)

        print("[%s] Train finished. Elapsed: %s"
                % (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    datetime.timedelta(seconds=time.time() - start_t)))

    def save(self, filename):
        model = self.model.state_dict()
        torch.save({'model': model}, filename)

    def get_tensor(self, fn):
      # load audio
      if self.dataset == 'mtat':
          npy_path = os.path.join(self.data_path, 'mtat', 'npy', fn[:-3]) + 'npy'
      raw = np.load(npy_path, mmap_mode='r')

      # split chunk
      length = len(raw)
      hop = (length - self.input_length) // self.batch_size
      x = torch.zeros(self.batch_size, self.input_length)
      for i in range(self.batch_size):
          x[i] = torch.Tensor(raw[i*hop:i*hop+self.input_length]).unsqueeze(0)
      return x

In [ ]:
# coding: utf-8
import os
import numpy as np
from torch.utils import data

class AudioFolder(data.Dataset):
  def __init__(self, root, split, input_length=None):
    self.root = root
    self.split = split
    self.input_length = input_length
    self.get_songlist()
    if TAG_NUM == 30:
      self.binary = np.load('./../split/mtat/30_binary.npy')
    else:
      self.binary = np.load('./../split/mtat/46_binary.npy')
    # self.binary = np.load('./../split/mtat/binary.npy')
    self.count = 0;

  def __getitem__(self, index):
    npy, tag_binary, exist = self.get_npy(index)
    return npy.astype('float32'), tag_binary.astype('float32')

  def get_songlist(self):
    if self.split == 'TRAIN':
      if TAG_NUM == 30:
        self.fl = np.load('./../split/mtat/30_train.npy')
      else:
        self.fl = np.load('./../split/mtat/46_train.npy')
      # self.fl = np.load('./../split/mtat/train.npy')
    elif self.split == 'VALID':
      if TAG_NUM == 30:
        self.fl = np.load('./../split/mtat/30_valid.npy')
      else:
        self.fl = np.load('./../split/mtat/46_valid.npy')
      # self.fl = np.load('./../split/mtat/valid.npy')
    elif self.split == 'TEST':
      if TAG_NUM == 30:
        self.fl = np.load('./../split/mtat/30_test.npy')
      else:
        self.fl = np.load('./../split/mtat/46_test.npy')
      # self.fl = np.load('./../split/mtat/test.npy')
    else:
      print('Split should be one of [TRAIN, VALID, TEST]')

  def get_npy(self, index):
    ix, fn = self.fl[index].split('\t')
    npy_path = os.path.join(self.root, 'mtat', 'npy', fn[:-3]) + 'npy'
    if self.count % 50 == 0:
      print("\nOpen count:", self.count, "Current file", npy_path)
    self.count += 1
    npy, tag_binary = None, None
    exist = True
    npy = np.load(npy_path, mmap_mode='r')
    # random_idx = int(np.floor(np.random.random(1) * (len(npy)-self.input_length)))
    # npy = np.array(npy[random_idx:random_idx+self.input_length])
    npy = np.array(npy[-self.input_length:])
    tag_binary = self.binary[int(ix)]
    return npy, tag_binary, exist

  def __len__(self):
    return len(self.fl)

def get_audio_loader(root, batch_size, split='TRAIN', num_workers=0, input_length=None):
  data_loader = data.DataLoader(dataset=AudioFolder(root, split=split, input_length=input_length),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers)
  return data_loader

###Training

#####Code for training

In [ ]:
# training/main.py
crnn = CRNN()
musicnn = Musicnn()

def main(config):
    np.random.seed(SEED)
    # path for models
    if not os.path.exists(config["model_save_path"]):
        os.makedirs(config["model_save_path"])

    # import data loader

    # audio length
    if config["model_type"] == 'crnn':
        config["input_length"] = 29 * 16000
    elif config["model_type"] == 'musicnn':
        #config["input_length"] = 3 * 16000
        config["input_length"] = 29 * 16000

    # get data loder
    train_loader = get_audio_loader(config["data_path"],
                                    config["batch_size"],
									                  split='TRAIN',
                                    input_length=config["input_length"],
                                    num_workers=config["num_workers"])
    solver = Solver(train_loader, config)
    solver.train()

# if __name__ == '__main__':
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument('--num_workers', type=int, default=0)
parser.add_argument('--dataset', type=str, default='mtat')
parser.add_argument('--n_epochs', type=int, default=1) # 200
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--use_tensorboard', type=int, default=1)
parser.add_argument('--model_save_path', type=str, default='/content/drive/MyDrive/CS376/')
parser.add_argument('--model_load_path', type=str, default='.')
parser.add_argument('--data_path', type=str, default='/content/drive/MyDrive/CS376/music_src/audionautix/')
parser.add_argument('--log_step', type=int, default=20)
parser.add_argument("-f", "--file", required=False) 
parser.add_argument('--model_type', type=str, default=MODEL)

/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:508: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  "At least one mel filterbank has all zero values. "


_StoreAction(option_strings=['--model_type'], dest='model_type', nargs=None, const=None, default='musicnn', type=<class 'str'>, choices=None, help=None, metavar=None)

#####Start training

In [ ]:
# current directory position (!pwd) should be like below
# content/drive/MyDrive/CS376/split

# use cd to move to proper directory
# !pwd
%cd ./drive/MyDrive/CS376/split

config = vars(parser.parse_args())
main(config)

/content/drive/MyDrive/CS376/split
start training with musicnn model...

Open count: 0 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/HipHop1_29_58.npy

Open count: 50 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/DatGrooveFullTrack.npy

Open count: 100 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/AMomentsReflection_58_87.npy

Open count: 150 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/90SecondsOfFunk_58_87.npy

Open count: 200 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/RightOnRedFull_29_58.npy

Open count: 250 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/Clouds_58_87.npy

Open count: 300 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/Essence2.npy

Open count: 350 Current file /content/drive/MyDrive/CS376/music_src/audionautix/mtat/npy/TheBigHouse_29_58.npy

Open count: 400 Current file

###Evaluation

#####Code for evaluation

In [ ]:
if TAG_NUM == 30:
  TAGS = ['calming' 'relaxing' 'uplifting' 'bright' 'grooving' 'energy' 'sad'
          'driving' 'bouncy' 'dark' 'pensive' 'somber' 'epic' 'cool' 'melancholy'
          'mysterious' 'moody' 'meditation' 'intense' 'humorous' 'suspenseful'
          'christmas' 'action' 'aggressive' 'dramatic' 'soothing' 'happy' 'angry'
          'ominous' 'jazz']
else:
  TAGS = ['calming', 'relaxing', 'uplifting', 'bright', 'grooving', 'energy', 'sad',
          'driving', 'bouncy', 'dark', 'pensive', 'somber', 'epic', 'cool',
          'guest-composer', 'groove', 'meloncholy', 'mysterious', 'moody', 'meditative',
          'intense', 'humorous', 'short-bits', 'suspenseful', 'christmas', 'action',
          'aggressive', 'meditation', 'melancholy', 'dramatic', 'vocal', 'dance',
          'soothing', 'light', 'happy', 'angry', 'ominous', 'evil', 'inspiration',
          'bird-house-music', 'goofy', 'silly', 'smooth', 'chill', 'bossa-nova', 'jazz']
        
def read_file(tsv_file):
    tracks = {}
    with open(tsv_file) as fp:
        reader = csv.reader(fp, delimiter='\t')
        next(reader, None)  # skip header
        for row in reader:
            track_id = row[0]
            tracks[track_id] = {
                'path': row[3].replace('.wav', '.npy'),
                'tags': row[5:],
            }
    return tracks

class Predict(object):
    def __init__(self):
        self.model_type = MODEL
        self.model_load_path = '/content/drive/MyDrive/CS376/training/model.py'
        self.dataset = 'mtat'
        self.data_path = '/content/drive/MyDrive/CS376/music_src/audionautix'
        self.batch_size = 16
        self.build_model()
        self.get_dataset()

    def get_model(self):
        if self.model_type == 'crnn':
            self.input_length = 29 * 16000
            return crnn
        elif self.model_type == 'musicnn':
            #self.input_length = 3 * 16000
            self.input_length = 29 * 16000
            return musicnn
        else:
            print('model_type has to be one of [musicnn, crnn]')

    def build_model(self):
        self.model = self.get_model()

    def get_dataset(self):
        if self.dataset == 'mtat':
          if TAG_NUM == 30:
              self.valid_list = np.load('./../split/mtat/30_valid.npy')
              self.test_list = np.load('./../split/mtat/30_test.npy')
              self.binary = np.load('./../split/mtat/30_binary.npy')
          else:
              self.valid_list = np.load('./../split/mtat/46_valid.npy')
              self.test_list = np.load('./../split/mtat/46_test.npy')
              self.binary = np.load('./../split/mtat/46_binary.npy')
          # small-size data with 46 tags
              # self.valid_list = np.load('./../split/mtat/valid.npy')
              # self.test_list = np.load('./../split/mtat/test.npy')
              # self.binary = np.load('./../split/mtat/binary.npy')

    def load(self, filename):
        print("file load", filename)
        S = torch.load(filename)
        if 'spec.mel_scale.fb' in S.keys():
            self.model.spec.mel_scale.fb = S['spec.mel_scale.fb']
        self.model.load_state_dict(S)

    def to_var(self, x):
        return Variable(x)

    def get_tensor(self, fn):
        # load audio
        if self.dataset == 'mtat':
            npy_path = os.path.join(self.data_path, 'mtat', 'npy', fn[:-3]) + 'npy'
        raw = np.load(npy_path, mmap_mode='r')

        # split chunk
        length = len(raw)
        hop = (length - self.input_length) // self.batch_size
        x = torch.zeros(self.batch_size, self.input_length)
        for i in range(self.batch_size):
            x[i] = torch.Tensor(raw[i*hop:i*hop+self.input_length]).unsqueeze(0)
        return x

    def get_auc(self, est_array, gt_array):
        accuracy = []
        precision = []
        recall = []
        f1 = []
        roc = []
        pr = []

        hit_count = 0
        for i in range(len(gt_array)):
          hit = 0
          y_true = gt_array[i].astype(np.float32)
          y_true = y_true.astype(np.int32)
          y_pred = est_array[i]
          # ranks = rankdata(y_pred)
          for j in range(len(y_pred)):
            # print(y_pred[j], y_true[j])
            if y_pred[j] > 0.52:
              y_pred[j] = 1
              if y_true[j] == 1:
                hit = 1
            else:
              y_pred[j] = 0

          hit_count += hit
          y_pred = y_pred.astype(np.int32)
          # using confusion matrix
          accuracy_score = metrics.accuracy_score(y_true, y_pred)
          precision_score = metrics.precision_score(y_true, y_pred, average='macro', zero_division=1)
          recall_score = metrics.recall_score(y_true, y_pred, average='macro', zero_division=1)
          f1_score = metrics.f1_score(y_true, y_pred, average='macro', zero_division=1)

          accuracy.append(accuracy_score)
          precision.append(precision_score)
          recall.append(recall_score)
          f1.append(f1_score)

        # print("HIT COUNT:", hit_count, " out of", len(gt_array))
        return np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(f1)

    def valid(self):
        accuracy_score, precision_score, recall_score, f1_score, loss = self.get_test_score(1)
        print('loss: %.4f' % loss)
        print('accuracy_score: %.4f' % accuracy_score)    # 전체 중 모델이 바르게 분류한 비율
        print('precision_score: %.4f' % precision_score)  # 모델이 positive라 분류한 것 중 실제 값이 positive인 비율
        print('recall_score: %.4f' % recall_score)        # 실제 값이 positive인 것 중 모델이 positivie라 분류한 비율
        print('f1_score: %.4f' % f1_score)                # precision과 recall의 조화평균

    def test(self):
        accuracy_score, precision_score, recall_score, f1_score, loss = self.get_test_score(2)
        print('loss: %.4f' % loss)
        print('accuracy_score: %.4f' % accuracy_score)    # 전체 중 모델이 바르게 분류한 비율
        print('precision_score: %.4f' % precision_score)  # 모델이 positive라 분류한 것 중 실제 값이 positive인 비율
        print('recall_score: %.4f' % recall_score)        # 실제 값이 positive인 것 중 모델이 positivie라 분류한 비율
        print('f1_score: %.4f' % f1_score)                # precision과 recall의 조화평균

    def get_test_score(self, vort):
        self.model = self.model.eval()
        est_array = []
        gt_array = []
        losses = []
        reconst_loss = nn.BCEWithLogitsLoss()
        if vort == 1:
          print("start validation ...")
          run_list = self.valid_list
        else:
          print("start testing ...")
          run_list = self.test_list

        for line in tqdm.tqdm(run_list):
            if self.dataset == 'mtat':
                ix, fn = line.split('\t')

            # load and split
            x = self.get_tensor(fn)

            # ground truth
            if self.dataset == 'mtat':
                ground_truth = self.binary[int(ix)]

            # forward
            x = self.to_var(x)
            y = torch.tensor([ground_truth.astype(np.float32) for i in range(self.batch_size)]) #.cuda()
            out = self.model(x)
            loss = reconst_loss(out, y)
            losses.append(float(loss.data))
            out = out.detach().cpu()

            # estimate
            estimated = np.array(out).mean(axis=0)
            est_array.append(estimated)
            gt_array.append(ground_truth)

        est_array, gt_array = np.array(est_array), np.array(gt_array)
        loss = np.mean(losses)

        accuracy_score, precision_score, recall_score, f1_score = self.get_auc(est_array, gt_array)
        return accuracy_score, precision_score, recall_score, f1_score, loss

#####Run validation or test, whatever you want

In [ ]:
p = Predict()

# validation
p.valid()

# test
p.test()

start validation ...


  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:174: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
100%|██████████| 100/100 [12:54<00:00,  7.75s/it]


loss: 0.8958
accuracy_score: 0.7837
precision_score: 0.5443
recall_score: 0.5767
f1_score: 0.5368
start testing ...


100%|██████████| 100/100 [13:07<00:00,  7.87s/it]


loss: 0.8998
accuracy_score: 0.7683
precision_score: 0.5273
recall_score: 0.5607
f1_score: 0.5200


###Prediction

Run this cell to load the song that you want to predict the tags.

Upload widget is only available when the cell has been executed in the current browser session.

In [ ]:
SAMPLE_RATE = 16000
DATASET = "mtat"
MODEL_NAMES = {
    "CRNN": "crnn",
    "MusiCNN": "musicnn"
}

class Predictor():
    def predict(self, input, variant, output_format):
        key = MODEL
        if TAG_NUM == 30:
          self.tags = np.load("./mtat/30_tags.npy")
        else:
          self.tags = np.load("./mtat/tags.npy")
        
        if variant == 'crnn':
          model = crnn.eval()
          input_length = 29 * 16000
        elif variant == "musicnn":
          model = musicnn.eval()
          #input_length = 3 * 16000
          input_length = 29 * 16000
        else:
            print('model_type has to be one of [musicnn, crnn]')

        signal, _ = librosa.core.load(str(input), sr=SAMPLE_RATE)
        length = len(signal)
        hop = length // 2 - input_length // 2
        x = torch.zeros(1, input_length)
        x[0] = torch.Tensor(signal[hop : hop + input_length]).unsqueeze(0)
        x = Variable(x)
        out = model(x)
        result = dict(zip(self.tags, out[0].detach().numpy().tolist()))

        # return result
        result_list = list(sorted(result.items(), key=lambda x: x[1]))
        plt.figure(figsize=[5, 10])
        plt.barh(
            np.arange(len(result_list)), [r[1] for r in result_list], align="center"
        )
        plt.yticks(np.arange(len(result_list)), [r[0] for r in result_list])
        plt.tight_layout()

        print("TAGS of top 5 probability: ")
        for i in range(5):
          print("rank", i+1, ": ", result_list[-i-1][0])

        return result

        # out_path = Path(tempfile.mkdtemp()) / "out.png"
        # plt.savefig(out_path)
        # return out_path

# current directory position (!pwd) should be like below
# content/drive/MyDrive/CS376/split

def _recognize(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  wavfile.write('test.wav', SAMPLE_RATE, (32767*audio).astype(np.int16))

  predictor = Predictor()
  predictor.predict('/content/drive/MyDrive/CS376/split/test.wav',
                 MODEL, 'JSON')


def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _recognize(audio)

try:
  _upload_audio("")
except TypeError:
  print("uploading failed")